In [ ]:
%load_ext autoreload
%autoreload 2
from path_setup import setup_project_root
root = setup_project_root()

import pandas as pd
import numpy as np
from datetime import datetime
from typing import List, Dict, Any

# Local modules
from etl_showcase.infrastructure.utils.file_utils import (
    dataframe_to_excel,
)
from etl_showcase.infrastructure.statistics.regression_utils import (
    run_multivariate_regression,
    display_regression_coefficients,
    get_regression_coefficients,
    generate_regression_predictions,
)
from etl_showcase.infrastructure.reporting.regression_visualizer import (
    plot_residual_analysis,
    plot_actual_vs_predicted,
)

def generate_simulated_data(
    index_column: str,
    x_columns: List[str],
    y_column: str,
    num_samples: int = 200,
) -> pd.DataFrame:
    """生成模擬的房屋數據，用於演示迴歸分析。"""
    print(f"--- 正在生成 {num_samples} 筆模擬資料 ---")

    # 模擬基礎自變數 (X variables)
    population = np.random.randint(500, 50000, num_samples)
    households = population * np.random.uniform(0.3, 0.45, num_samples)
    total_rooms = population * np.random.uniform(1.0, 3.0, num_samples)
    total_bedrooms = total_rooms * np.random.uniform(0.15, 0.25, num_samples)
    median_income = np.random.randint(25000, 150000, num_samples) 

    # --- 新增：地理位置與屋齡模擬 ---
    # 模擬經緯度 (以台灣大致範圍為例)
    longitude = np.random.uniform(120.0, 122.0, num_samples)
    latitude = np.random.uniform(22.0, 25.0, num_samples)
    
    longitude_sq = longitude ** 2  # 房屋經度平方
    latitude_sq = latitude ** 2    # 房屋緯度平方
    house_age = np.random.randint(1, 51, num_samples)  # 街區房屋年齡中位數 (1-50年)

    # 模擬應變數 (Y variable) - 房屋價值中位數
    # 納入新變數的影響：屋齡越高通常房價越低
    median_house_value = (
        50000  # 基礎價值
        + median_income * 2.5 
        + total_rooms * 50  
        + total_bedrooms * 200  
        + (population / 1000) * 500  
        - house_age * 1500  # 屋齡每增加一年，價值減少1500 (負相關)
        + (longitude_sq / 100) * 10 # 地理位置的微弱非線性影響
        + np.random.normal(0, 40000, num_samples)  # 隨機噪音
    )
    
    # 確保價值不會低於一萬
    median_house_value[median_house_value < 10000] = 10000

    # 索引欄位 (1 到 N)
    indices = np.arange(1, num_samples + 1)

    # 組合 DataFrame
    data = pd.DataFrame({
        index_column: indices,
        y_column: median_house_value,
        x_columns[0]: longitude_sq,
        x_columns[1]: latitude_sq,
        x_columns[2]: median_income,
        x_columns[3]: total_rooms.astype(int),
        x_columns[4]: total_bedrooms.astype(int), 
        x_columns[5]: population,
        x_columns[6]: house_age
    })

    return data

def regression_analysis(
    data: pd.DataFrame,
    index_column: str,
    x_columns: List[str],
    y_column: str,
    result_file_postfix: str,
):   
    # 1. 執行迴歸
    regression_results = run_multivariate_regression(
        data = data, 
        x_columns = x_columns,
        y_column = y_column,
    )

    # 2. 匯出迴歸係數 Excel 檔案
    print(f"\n--- 正在把迴歸係數相關資訊匯出成 Excel 檔案 ---")
    coef_df = get_regression_coefficients(regression_results)
    dataframe_to_excel(
        df = coef_df,
        file_path = f'data/results/迴歸係數相關資訊_{result_file_postfix}.xlsx',
        keep_index = True,
    )   

    # 3. 匯出預測值 Excel 檔案
    print(f"\n--- 正在計算預測值並匯出 Excel 檔案 ---")
    predictions_df = generate_regression_predictions(
        df_original = data, 
        results = regression_results,
        index_column = index_column,
        x_columns = x_columns,
        y_column = y_column,
    )
    dataframe_to_excel(
        df = predictions_df,
        file_path = f'data/results/迴歸預測結果_{result_file_postfix}.xlsx',
    )

    # 4. 畫殘差圖、實際值 vs. 預測值散點圖供檢查模型可靠性
    print(f"\n--- 透過圖檢查模型可靠性 ---")
    plot_residual_analysis(
        df = predictions_df, 
        actual_col = 'true values', 
        pred_col = 'predicted values',
    )
    plot_actual_vs_predicted(
        df = predictions_df, 
        actual_col = 'true values', 
        pred_col = 'predicted values',
    )    

# --- 主程式執行區塊 ---
if __name__ == "__main__":
    # 定義共用變數
    index_column = '索引'
    y_column = '街區房屋價值中位數log值'
    x_columns = ['房屋經度平方','房屋緯度平方','街區家庭收入中位數log值', '每戶平均房間數', '每戶平均臥室數', '每戶平均人口數','街區房屋年齡中位數']   
    
    # 1. 載入或生成資料
    try:
        # 從實際 Excel 檔案載入
        # data = pd.read_excel('data/raw/房價資料範例.xlsx')
        
        # 模擬資料
        data = generate_simulated_data(
            index_column = index_column,
            x_columns = x_columns,
            y_column = y_column,
        )
    except FileNotFoundError:
        print("找不到指定的 Excel 檔案，請檢查路徑。")
        exit()
    except Exception as e:
        print(f"載入或處理資料時發生錯誤: {e}")
        exit()

    # 2. 進行一次迴歸分析
    date_str = datetime.now().strftime("%Y%m%d")
    print('==========分析資料==========')
    regression_analysis(
        data = data,
        index_column = index_column,
        x_columns = x_columns,
        y_column = y_column,
        result_file_postfix = f'{date_str}_v1',
    )  